In [3]:
import tensorflow as tf
from tensorflow import keras
from keras import datasets, layers, models
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import precision_score, recall_score
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import random
import os

In [6]:
dataset_dir = '/Users/MarcelM/Documents/GitHub/LungsXray-FP/data/raw/SOFTMAX'

In [9]:
image_size = (150, 150)
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2,
    rotation_range=15,  # Nueva técnica: rotación
    width_shift_range=0.1,  # Nueva técnica: desplazamiento horizontal
    height_shift_range=0.1,  # Nueva técnica: desplazamiento vertical
)
train_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)
validation_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 5266 images belonging to 4 classes.
Found 1315 images belonging to 4 classes.


In [12]:
num_classes = len(train_generator.class_indices)
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),  # Reducir las neuronas aquí también
    tf.keras.layers.Dropout(0.5),  # Mantener el Dropout para reducir el sobreajuste
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [13]:
epochs = 10

history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator
)

Epoch 1/10
165/165 [==============================] - 353s 2s/step - loss: 1.0335 - accuracy: 0.5517 - val_loss: 0.7389 - val_accuracy: 0.6989
Epoch 2/10
165/165 [==============================] - 338s 2s/step - loss: 0.7503 - accuracy: 0.7058 - val_loss: 0.6146 - val_accuracy: 0.7650
Epoch 3/10
165/165 [==============================] - 337s 2s/step - loss: 0.6641 - accuracy: 0.7440 - val_loss: 0.6057 - val_accuracy: 0.7795
Epoch 4/10
165/165 [==============================] - 336s 2s/step - loss: 0.6081 - accuracy: 0.7683 - val_loss: 0.5560 - val_accuracy: 0.7894
Epoch 5/10
165/165 [==============================] - 320s 2s/step - loss: 0.5655 - accuracy: 0.7862 - val_loss: 0.4813 - val_accuracy: 0.8129
Epoch 6/10
165/165 [==============================] - 345s 2s/step - loss: 0.5384 - accuracy: 0.8029 - val_loss: 0.4705 - val_accuracy: 0.8243
Epoch 7/10
165/165 [==============================] - 343s 2s/step - loss: 0.5036 - accuracy: 0.8141 - val_loss: 0.4558 - val_accuracy: 0.8342

In [14]:
test_loss, test_acc = model.evaluate(validation_generator, verbose=2)
print('\nExactitud en el conjunto de validación:', test_acc)

42/42 - 67s - loss: 0.4060 - accuracy: 0.8471 - 67s/epoch - 2s/step

Exactitud en el conjunto de validación: 0.8471482992172241


In [15]:
model.save('modelo_softmax_invertido.keras')

In [17]:
model = load_model('/Users/MarcelM/Documents/GitHub/LungsXray-FP/models/modelo_softmax_invertido.keras')  # Ajusta el nombre de tu modelo

# Preprocesamiento de datos para el conjunto de prueba
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    dataset_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',  # Ajusta si tu modelo espera etiquetas categóricas
    shuffle=False
)

# Obtener las predicciones del modelo para el conjunto de prueba
predictions = model.predict(test_generator)

# Convertir las predicciones a clases (si es necesario)
predicted_classes = np.argmax(predictions, axis=1)

# Obtener las etiquetas verdaderas
true_classes = test_generator.classes

# Calcular la precisión y el recall por clase
classification_rep = classification_report(true_classes, predicted_classes)
accuracy = accuracy_score(true_classes, predicted_classes)
conf_matrix = confusion_matrix(true_classes, predicted_classes)

print("Informe de clasificación:")
print(classification_rep)

print("Exactitud general:")
print(accuracy)

print("Matriz de confusión:")
print(conf_matrix)

Found 6581 images belonging to 4 classes.
206/206 [==============================] - 212s 1s/step
Informe de clasificación:
              precision    recall  f1-score   support

           0       0.95      0.82      0.88      1589
           1       0.85      0.93      0.89      1341
           2       0.95      0.92      0.93      2616
           3       0.80      0.95      0.87      1035

    accuracy                           0.90      6581
   macro avg       0.89      0.90      0.89      6581
weighted avg       0.91      0.90      0.90      6581

Exactitud general:
0.9012308159854125
Matriz de confusión:
[[1309   16   62  202]
 [  10 1246   58   27]
 [   9  192 2396   19]
 [  43    5    7  980]]


In [19]:
from tensorflow.keras.utils import img_to_array, load_img

# Lista de carpetas en el directorio
folders = os.listdir(dataset_dir)

# Modelo preentrenado
model = load_model('/Users/MarcelM/Documents/GitHub/LungsXray-FP/models/modelo_softmax_invertido.keras')

# Número de imágenes a seleccionar por carpeta
num_images_por_carpeta = 25

# Crear un diccionario que mapea índices a nombres de carpetas
index_to_folder = {i: folder for i, folder in enumerate(folders)}

# Iterar sobre cada carpeta
for folder in folders:
    folder_path = os.path.join(dataset_dir, folder)
    
    # Lista de archivos en la carpeta
    files = os.listdir(folder_path)
    
    # Seleccionar aleatoriamente 25 imágenes
    selected_files = random.sample(files, num_images_por_carpeta)
    
    # Iterar sobre cada imagen seleccionada
    for file in selected_files:
        file_path = os.path.join(folder_path, file)
        
        # Cargar la imagen y preprocesarla
        img = load_img(file_path, target_size=(150, 150))
        img_array = img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array /= 255.0  # Normalizar
        
        # Hacer la predicción con el modelo
        prediction = model.predict(img_array)
        
        # Obtener la clase predicha
        predicted_class = np.argmax(prediction)
        
        # Obtener el nombre de la carpeta asociada a la clase predicha
        predicted_folder = index_to_folder[predicted_class]
        
        # Mostrar resultados
        print(f"Imagen: {file}, Carpeta Real: {folder}, Carpeta Predicha: {predicted_folder}")

1/1 [==============================] - 0s 177ms/step
Imagen: Tuberculosis-412.png, Carpeta Real: TUBERCULOSIS, Carpeta Predicha: COVID
1/1 [==============================] - 0s 29ms/step
Imagen: Tuberculosis-272.png, Carpeta Real: TUBERCULOSIS, Carpeta Predicha: COVID
1/1 [==============================] - 0s 27ms/step
Imagen: Tuberculosis-325.png, Carpeta Real: TUBERCULOSIS, Carpeta Predicha: COVID
1/1 [==============================] - 0s 30ms/step
Imagen: Tuberculosis-559.png, Carpeta Real: TUBERCULOSIS, Carpeta Predicha: COVID
1/1 [==============================] - 0s 29ms/step
Imagen: CHNCXR_0403_1.png, Carpeta Real: TUBERCULOSIS, Carpeta Predicha: COVID
1/1 [==============================] - 0s 31ms/step
Imagen: CHNCXR_0557_1.png, Carpeta Real: TUBERCULOSIS, Carpeta Predicha: COVID
1/1 [==============================] - 0s 35ms/step
Imagen: CHNCXR_0654_1.png, Carpeta Real: TUBERCULOSIS, Carpeta Predicha: COVID
1/1 [==============================] - 0s 29ms/step
Imagen: Tuberculos